In [1]:
import bempp.api
import numpy as np

import sys
sys.path.append('../../')
from decimal import *

import bemppUQ
from bemppUQ.operators.maxwell import assemble_operators, evaluate_far_field, evaluate_far_field_sd
from bemppUQ.shapes import kite, perturbate
from bempp.api.assembly.blocked_operator import (
    coefficients_of_grid_function_list,
    projections_of_grid_function_list,
    grid_function_list_from_coefficients,
)

from bemppUQ.foa.operators import (
    function_product,
    surface_divergence,
    surface_gradient,
    trace_transformation,
)

bempp.api.global_parameters.assembly.potential_operator_assembly_type = 'dense'
#bempp.api.global_parameters.assembly.boundary_operator_assembly_type = 'dense'
bempp.api.global_parameters.hmat.eps = 1E-4

config = bemppUQ.config.set_case('A')
precision = 20

h = 2.0 * np.pi / (precision * config["k_int"])

print(h)
base_grid = kite(h=h) 
print(base_grid.leaf_view.entity_count(1))

grid_eta, grid_fun = perturbate(base_grid, 0)

# far_field for t=0

transmission_operators = assemble_operators(base_grid, config)
print('FF')
far_field, solution = evaluate_far_field(transmission_operators, config)
print('FFp')
far_field_p, sol_p = evaluate_far_field_sd(base_grid, transmission_operators, config, solution, grid_fun)

0.07226349639084238
9003
FF
iteration - 1 || residual - 0.311135766780663 7.680907964706421
iteration - 2 || residual - 0.10666263809412584 2.6807057857513428
iteration - 3 || residual - 0.036725492202178446 2.6657731533050537
iteration - 4 || residual - 0.012515960412134116 2.648833751678467
iteration - 5 || residual - 0.004096972587749192 2.6534695625305176
iteration - 6 || residual - 0.0013536033830406798 2.2825586795806885
iteration - 7 || residual - 0.00044658851591589707 2.2438158988952637
iteration - 8 || residual - 0.00014721985394386644 2.254467487335205
iteration - 9 || residual - 5.1381625874714985e-05 2.2324910163879395
iteration - 10 || residual - 1.778789145163166e-05 2.478891372680664
iteration - 11 || residual - 6.128064052640806e-06 2.663302183151245
FFp
iteration - 1 || residual - 0.2563075532341251 7.926316976547241
iteration - 2 || residual - 0.09429598855585572 2.623199701309204
iteration - 3 || residual - 0.03037278780486346 2.5371294021606445
iteration - 4 || res

In [2]:
far_field_eta_list = []

t_list = [1E-6, 1E-5, 1E-4, 1E-3, 0.01, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 1.5, 2]


config["spaces"] = "maxwell_primal"  # or maxwell
config["osrc"] = True

for t in t_list:
    print(t, 't now')
    grid_eta, _ = perturbate(base_grid, t)
    transmission_operators_eta = assemble_operators(grid_eta, config)
    far_field_eta, _ = evaluate_far_field(transmission_operators_eta, config)
    
    far_field_eta_list.append(far_field_eta)
    
    residual_eta = far_field_eta - far_field
    residual_p = far_field_eta - far_field - t * far_field_p
    
    err_0 = np.linalg.norm(residual_eta)/np.linalg.norm(far_field_eta)
    err_FOA = np.linalg.norm(residual_p)/ np.linalg.norm(far_field_eta)
    
    print(err_0, 'relative norm for the residual eta')
    print(err_FOA, 'norm for the residual first order')
    
    


1e-06 t now
iteration - 1 || residual - 0.3508267799224139 0.6834812164306641
iteration - 2 || residual - 0.17321793402666863 0.22059917449951172
iteration - 3 || residual - 0.11300145444261228 0.21579504013061523
iteration - 4 || residual - 0.06117226781232967 0.2208247184753418
iteration - 5 || residual - 0.03616883739263946 0.21428775787353516
iteration - 6 || residual - 0.021082568469228905 0.21490097045898438
iteration - 7 || residual - 0.011364171745824152 0.21861815452575684
iteration - 8 || residual - 0.00667528787330023 0.21243786811828613
iteration - 9 || residual - 0.0037995366385361867 0.21065068244934082
iteration - 10 || residual - 0.002187410652680777 0.21177911758422852
iteration - 11 || residual - 0.0012531481996667071 0.23033857345581055
iteration - 12 || residual - 0.0008153678677503131 0.19510602951049805
iteration - 13 || residual - 0.0005555215864101352 0.20806646347045898
iteration - 14 || residual - 0.00039106625108559104 0.19096851348876953
iteration - 15 || re

iteration - 26 || residual - 1.835840439643054e-05 0.22144770622253418
iteration - 27 || residual - 1.4510445188119294e-05 0.22152972221374512
iteration - 28 || residual - 1.132425841431981e-05 0.19197821617126465
iteration - 29 || residual - 8.86868599664804e-06 0.19896602630615234
0.0058802229205167445 relative norm for the residual eta
0.006012903220045631 norm for the residual first order
0.01 t now
iteration - 1 || residual - 0.34932251281874166 0.7145051956176758
iteration - 2 || residual - 0.17253830842603401 0.2429800033569336
iteration - 3 || residual - 0.11243265047874054 0.2305140495300293
iteration - 4 || residual - 0.06062384956541057 0.2058098316192627
iteration - 5 || residual - 0.035694243432051846 0.20692086219787598
iteration - 6 || residual - 0.02074241628835736 0.2064673900604248
iteration - 7 || residual - 0.011177842506726193 0.2087116241455078
iteration - 8 || residual - 0.006524216017067924 0.2239062786102295
iteration - 9 || residual - 0.003726236535676504 0.20

iteration - 21 || residual - 5.1516652043858976e-05 0.26080989837646484
iteration - 22 || residual - 4.2387600794845615e-05 0.27851319313049316
iteration - 23 || residual - 3.404562612510148e-05 0.27520275115966797
iteration - 24 || residual - 2.70591053659429e-05 0.3466837406158447
iteration - 25 || residual - 2.160542175797647e-05 0.3370189666748047
iteration - 26 || residual - 1.7034472847358663e-05 0.25537586212158203
iteration - 27 || residual - 1.3467763421746302e-05 0.24125266075134277
iteration - 28 || residual - 1.0430535550570405e-05 0.1945028305053711
iteration - 29 || residual - 8.153567274277036e-06 0.2794952392578125
0.03939793617938151 relative norm for the residual eta
0.01224787206876903 norm for the residual first order
0.05 t now
iteration - 1 || residual - 0.3436306566039047 0.6665802001953125
iteration - 2 || residual - 0.17003146617428525 0.21919798851013184
iteration - 3 || residual - 0.11037173908176934 0.2107996940612793
iteration - 4 || residual - 0.0586504112

iteration - 20 || residual - 4.482684634673063e-05 0.5317533016204834
iteration - 21 || residual - 3.51730697771243e-05 0.2433314323425293
iteration - 22 || residual - 2.8875452242376598e-05 0.2622840404510498
iteration - 23 || residual - 2.27821219112795e-05 0.2699151039123535
iteration - 24 || residual - 1.773563810460493e-05 0.33332395553588867
iteration - 25 || residual - 1.395392374727721e-05 0.3015098571777344
iteration - 26 || residual - 1.0854698658839478e-05 0.2266702651977539
iteration - 27 || residual - 8.468165289806921e-06 0.29956889152526855
0.6165319273123947 relative norm for the residual eta
0.525866328908296 norm for the residual first order
1 t now
iteration - 1 || residual - 0.3387888787920587 0.7017884254455566
iteration - 2 || residual - 0.17789641520522634 0.23885321617126465
iteration - 3 || residual - 0.11502089128592323 0.2241649627685547
iteration - 4 || residual - 0.06519120729075166 0.2689673900604248
iteration - 5 || residual - 0.03800937538935886 0.245005

In [3]:
name = (
    "results/"
        + "precision"
    + str(precision))


my_dict = {
        "far_field": far_field,
    "far_field_p": far_field_p,
    "far_field_eta": far_field_eta_list,
    }

np.save(name + ".npy", my_dict)    

In [12]:
i = 0
for t in t_list:
    far_field_eta = my_dict['far_field_eta'][i]
    
    far_field_eta_list.append(far_field_eta)
    
    residual_eta = far_field_eta - far_field
    residual_p = far_field_eta - far_field - t * far_field_p
    
    err_0 = np.linalg.norm(residual_eta)/np.linalg.norm(far_field_eta)
    err_FOA = np.linalg.norm(residual_p)/ np.linalg.norm(far_field_eta)
    
    print(t,err_0, err_FOA, '0 | 1')
    i += 1

1e-06 0.005991650657686997 0.005991983530624762 0 | 1
1e-05 0.005988306695381757 0.005991618341798641 0 | 1
0.0001 0.005958819533086344 0.005990210952727774 0 | 1
0.001 0.0058802229205167445 0.006012903220045631 0 | 1
0.01 0.016035879285042375 0.007384594763075914 0 | 1
0.01 0.016035879285036973 0.007384594763087162 0 | 1
0.02 0.03152257476635535 0.010441737413990322 0 | 1
0.025 0.03939793617938151 0.01224787206876903 0 | 1
0.05 0.07868818486500188 0.022594675486151388 0 | 1
0.1 0.15497404360004782 0.04887124337849375 0 | 1
0.25 0.35663822339130036 0.18267110745263554 0 | 1
0.5 0.6165319273123947 0.525866328908296 0 | 1
1 1.0302211835179802 1.4279233610917548 0 | 1
1.5 1.2038113915545687 2.355098639496757 0 | 1
2 1.355427125466834 3.22985350416376 0 | 1


In [9]:
my_dict[]

[1e-06,
 1e-05,
 0.0001,
 0.001,
 0.01,
 0.01,
 0.02,
 0.025,
 0.05,
 0.1,
 0.25,
 0.5,
 1,
 1.5,
 2]